In [5]:
import json
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer as tw_tokenizer
from collections import Counter
from unidecode import unidecode

print("Operasi dimulai...")
tweets_data = [] # Membuat list kosong untuk menyimpan data json perbaris
tweets_file = open('crawling_ke_1_tgl_4April2019.txt', "r") # membuka file
for line in tweets_file:
    try:
        tweet = json.loads(line)  # Membaca data dalam format json dari file perbaris
        tweets_data.append(tweet) # Menambahkan data dari file ke dalam list
    except:
        continue
tweets_file.close()

stopword_file1 = open('TUGAS_3/stopword/20190327_stopword_id.txt', "r").read() # Membuka file stopword bahasa indonesia dan menjadikan isi file tersebut sebagai string
stopword_file2 = open('TUGAS_3/stopword_en/20190327_stopwords_en.txt', "r").read()  # Membuka file stopword bahasa inggris dan menjadikan isi file tersebut sebagai string
stopword_file3 = open('TUGAS_3/stopword_noise/20190327_stopword_noise.txt', "r").read()  # Membuka file stopword noise dan menjadikan isi file tersebut sebagai string
stopword_file_all = stopword_file1 + stopword_file2 + stopword_file3  # Menggabungkan ketiga string stopword sebelumnya kedalam satu string
stopwords = stopword_file_all.split('\n') # Memisahkan kata dalam string yang sudah digambungkan berdasarkan baris

slangwords = dict() # Membuat dictionary kosong untuk menyimpan kata slang dan formal sebagai key dan value
slangwords_dataframe = pd.read_csv('colloquial-indonesian-lexicon.csv') # Membuka file csv yang berisi kata slang dan formal dan mengkonversi kedalam dataframe
for slang, formal in zip(slangwords_dataframe['slang'], slangwords_dataframe['formal']):
    slangwords[slang] = formal # Mapping kata slang dan formal dan memasukkan ke dalam dictionary secara berulang

slangword_file = open('TUGAS_3/slangword/20190327_slangword.txt', "r").read() # Membuka file yang berisi kata slang dan kata formal dan mengkonversi kedalam string
slangwords_text = slangword_file.split('\n') # Memisahkan kata berdasarkan baris namun kata slang dan kata formal masih belum terpisah. output : (['slang:formal', ...])
for slang in slangwords_text:
    split_slang = slang.split(":") # Memisahkan semua kata slang dan kata formal berdasarkan "titik dua (:)"
    slangwords[split_slang[0]] = split_slang[1] # Mapping semua kata slang dan kata formal ke dalam dictionary. Output : {'slang' : 'formal', ...}

tweets = pd.DataFrame()
tweets['user_id'] = list(map(lambda tweet: tweet['id_str'], tweets_data))
tweets['user_name'] = list(map(lambda tweet: tweet['user']['name'], tweets_data))
tweets['user_screen_name'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets_data))
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['address'] = list(map(lambda tweet: tweet['place']['full_name'] if tweet['place'] != None else None, tweets_data))
hashtags = list(map(lambda tweet: tweet['entities']['hashtags'] if tweet['entities'] != None else None, tweets_data))
tweets['hashtags'] = list(map(lambda tweet : ', '.join(list(map(lambda tw : tw['text'], tweet))), hashtags))
mentions = list(map(lambda tweet: tweet['entities']['user_mentions'] if tweet['entities'] != None else None, tweets_data))
tweets['mentions'] = list(map(lambda tweet : ', '.join(list(map(lambda tw : tw['screen_name'], tweet))), mentions))
tweets['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data))
tweets['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['country'] = list(map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data))
#tweets.to_csv('export_dataframe.csv', encoding='utf-8')
#tweets.to_excel('export_dataframe.xlsx')
print ("Jumlah data tweets :", len(tweets))
tweetsIn = tweets[tweets.lang == 'in']
tweetsIn.head()
print ("Jumlah data tweets bahasa Indonesia :", len(tweetsIn))

stopword_file1 = open('TUGAS_3/stopword/20190327_stopword_id.txt', "r").read() # Membuka file stopword bahasa indonesia dan menjadikan isi file tersebut sebagai string
stopword_file2 = open('TUGAS_3/stopword_en/20190327_stopwords_en.txt', "r").read()  # Membuka file stopword bahasa inggris dan menjadikan isi file tersebut sebagai string
stopword_file3 = open('TUGAS_3/stopword_noise/20190327_stopword_noise.txt', "r").read()  # Membuka file stopword noise dan menjadikan isi file tersebut sebagai string
stopword_file_all = stopword_file1 + stopword_file2 + stopword_file3  # Menggabungkan ketiga string stopword sebelumnya kedalam satu string
stopwords = stopword_file_all.split('\n') # Memisahkan kata dalam string yang sudah digambungkan berdasarkan baris

slangwords = dict() # Membuat dictionary kosong untuk menyimpan kata slang dan formal sebagai key dan value
slangwords_dataframe = pd.read_csv('colloquial-indonesian-lexicon.csv') # Membuka file csv yang berisi kata slang dan formal dan mengkonversi kedalam dataframe

for slang, formal in zip(slangwords_dataframe['slang'], slangwords_dataframe['formal']):
    slangwords[slang] = formal # Mapping kata slang dan formal dan memasukkan ke dalam dictionary secara berulang

slangword_file = open('TUGAS_3/slangword/20190327_slangword.txt', "r").read() # Membuka file yang berisi kata slang dan kata formal dan mengkonversi kedalam string
slangwords_text = slangword_file.split('\n') # Memisahkan kata berdasarkan baris namun kata slang dan kata formal masih belum terpisah. output : (['slang:formal', ...])

for slang in slangwords_text:
    split_slang = slang.split(":") # Memisahkan semua kata slang dan kata formal berdasarkan "titik dua (:)"
    slangwords[split_slang[0]] = split_slang[1] # Mapping semua kata slang dan kata formal ke dalam dictionary. Output : {'slang' : 'formal', ...}

array_text = list(map(lambda tweet: unidecode(tweet).lower(), tweets['text'])) # Mapping semua text twitter dan memasukan ke dalam list. Output : ['teks panjang ...', 'teks panjang', ...]
long_text = ' '.join(array_text) # Menggabungkan semua text yang berada dalam list kedalam satu text
tokenized = tw_tokenizer().tokenize(long_text) # Memisahkan kata dalam text berasarkan "spasi"
tokenized2 = [w for w in tokenized if w.isalnum()] # Filtering kata yang hanya berisi karakater a-z dan 0-9
tokenized3 = list(map(lambda w : slangwords[w] if w in slangwords.keys() else w, tokenized2)) # Mengubah kata slang menjadi kata formal (kata slang dan kata formal yang diperoleh dari dictionary) 
#tokenized2 = list(map(lambda w : slangwords[w] if w in slangwords.keys() else None, tokenized))
# print(tokenized2)
# for w in tokenized2:
#     if w in slangwords.keys():
#         print(slangwords[w])
# print("Slang word sudah diganti kata baku")
# print(tokenized3)

# print(slangwords['yg'])
remove_stopwords = [w for w in tokenized3 if w not in stopwords] # Filtering data dengan menghapus kata yang tidak bermakna (Stopword yang diperoleh dari file)
Counter  = Counter(remove_stopwords) # Menghitung frekuensi muncul semua kata
most_words = Counter.most_common(100) # Menghitung 100 kata yang paling banyak muncul (Tidak termasuk kata yang tidak bermakna)
print(most_words)

print("Operasi selesai")

Operasi dimulai...
Jumlah data tweets : 7083
Jumlah data tweets bahasa Indonesia : 5365
[('2', 386), ('jakarta', 191), ('pagi', 185), ('allah', 167), ('tol', 147), ('banget', 137), ('1', 121), ('jam', 111), ('selamat', 110), ('indonesia', 109), ('tidur', 102), ('beli', 88), ('pakai', 84), ('suka', 80), ('sms', 80), ('6287788719', 77), ('668', 77), ('2019', 75), ('cawang', 74), ('anak', 72), ('imam', 72), ('semoga', 71), ('ku', 68), ('mahdi', 68), ('5', 67), ('kasih', 66), ('jalan', 59), ('hati', 58), ('amin', 58), ('sayang', 58), ('3', 58), ('100', 57), ('april', 56), ('lihat', 56), ('kerja', 55), ('asli', 55), ('rumah', 54), ('cepat', 54), ('malam', 52), ('terima', 52), ('masuk', 50), ('kemarin', 50), ('makan', 49), ('lupa', 49), ('2013', 49), ('main', 48), ('yuk', 47), ('coba', 47), ('4', 47), ('kamis', 46), ('manusia', 45), ('foto', 45), ('membantu', 45), ('rambut', 45), ('morning', 45), ('semangat', 43), ('keren', 42), ('manfaat', 42), ('2016', 42), ('krim', 42), ('open', 41), ('sa